In [1]:
import pandas as pd
import json

In [20]:
import tqdm

In [21]:
import time
import re

In [ ]:
from mmw.parse import *

In [ ]:
all_parsed_essays = parse_essays()

In [ ]:
marked_essays, filtered_marked_essays, all_questions = parse_annotations()

In [ ]:
all_answers = []
for i,essay_array in tqdm.tqdm(enumerate(all_parsed_essays[:LIMIT_ESSAYS])):
    
    essay_id = int(re.findall(r'\d+', essay_array[0])[-1])
    cur_row = filtered_marked_essays[filtered_marked_essays['essay id'] == essay_id]
    if len(cur_row) != 1:
        continue

    for j,k in enumerate(all_questions):
        
        prompt = assemble_prompt(essay_array[3][:-1], #implicit newline
                                 str(k).strip())

        all_answers.append((i, j, prompt, None, int(cur_row[k])))

In [39]:
len(all_answers) # 676 took 5 minutes with text-davinci-003, cost 12$ / 7seconds,23ct per essay (13 questions)
#-> need to use chat gpt to save bucks!?

988

In [43]:
finetune_set = []
for a in all_answers:
    finetune_set.append({'prompt': a[2], 'completion': str(a[-1])})

In [44]:
finetune_set[0]

{'prompt': 'Given is a student essay and a question to the essay. Please rate the Question with a number between 0 and 3, where 0 is "not at all satisfied" and 3 is "very satisfied.".\n\nEssay: """\nThe Cage\nIt felt like I was walking forever, i didn\'t even know were i was i just knew i was somewhere. A creepy aliway to be exazaced. To be honest i would rather\nbe at school than this nightmare well not to brag but i was not scared in the slightited, anyway desided just to keep walking to see what was down this\naliway, just as i was almost there thunder shot right though the sky it was like someone had a shotgun shot a bullet right next to my ear jezz i was not\nprepared for that, where there is thunder there is rain and i was not gonna get my new guchi shoes wet so i bulted to the end of the crusty aliway,\ni saw... well i didn\'t see anything other than cages, well i guess it was just a dream cause i don\'t own\nguchi shoes. OK enough of me being broke I gotta get to school. Time f

In [47]:
json.dump(finetune_set, open('finetune_set.jsonl','w'))